## Quickstart: Getting Started with the Snowflake Python API


### Creating and managing Snowpark Container Services with the Snowflake Python API

This notebook contains **sample code** that runs an NGINX web server using Snowpark Container Services, all running in Snowflake. The notebook is provided for convenience and demonstrative purposes. If your Snowflake account is in one of the select AWS regions where Snowpark Container Services is in Public Preview, you can use the code in this notebook as reference to help you orchestrate Snowpark Container Services using the Snowflake Python API.

Create our connection to Snowflake, create a `Root` object, and establish references to existing Snowflake objects in the Snowflake account:

In [ ]:
from pprint import pprint
from snowflake.core import Root
from snowflake.core.service import ServiceSpecInlineText
from snowflake.snowpark import Session

session = Session.builder.config("connection_name", "python_api").create()
api_root = Root(session)
database = api_root.databases["spcs_python_api_db"]
schema = database.schemas["public"]

To run a service, we'll need to create a compute pool, which can be defined and created with the Snowflake Python API:

In [ ]:
new_compute_pool_def = ComputePool(
    name="MyComputePool"
    instance_family="CPU_X64_XS",
    min_nodes=1,
    max_nodes=2,
)

new_compute_pool = api_root.compute_pools.create(new_compute_pool_def)

The container needed lives in an image repository, listed as a stage in Snowflake. Create a reference to the image repository so that we can grab important information related to the container as we define our service (in subsequent cells).

In [ ]:
image_repository = schema.image_repositories["MyImageRepository"]


Let's create our service. A service refers to a collection of containers running in a compute pool, all orchestrated in Snowflake. Using a compute pool I’ve already setup, and my container image, we can start the service.

In [ ]:
from textwrap import dedent
from io import BytesIO
from snowflake.core.service import Service
session.use_role("my_role")  # Perhaps this role has permission to create and run a service
specification = dedent(f"""\
            spec:
              containers:
              - name: web-server
                image: {image_repository.fetch().repository_url}/nginx:latest
              endpoints:
              - name: ui
                port: 80
                public: true
             """)

service_def = Service(
    name="MyService",
    compute_pool="MyComputePool",
    spec=ServiceSpecInlineText(specification),
    min_instances=1,
    max_instances=1,
)

nginx_service = schema.services.create(service_def)

With the service created, the next cell will output the status of the service.

In [ ]:
pprint(nginx_service.get_service_status(timeout=5))

It'll take a few minutes for the endpoints that are needed to access the service to be provisioned. The next cell isn't specific to Snowpark Container Services or the Snowflake Python API – it simply provides a handy way to inspect whether the endpoints are ready. 

In [ ]:
# The server gives us an "in progress" status string while the service
# is provisioned, so we'll wait a bit for the service to be available (the
# result is a JSON parsable string). It can take up to
# two minutes to provision the service.
import json, time
while True:
    public_endpoints = nginx_service.fetch().public_endpoints
    try:
        endpoints = json.loads(public_endpoints)
    except json.JSONDecodeError:
        print(public_endpoints)
        time.sleep(15)
    else:
        break

Once the endpoints are provisioned, the next cell will open the public endpoints in your browser.

In [ ]:
print(f"Visiting {endpoints['ui']} in your browser. You may need to log in there.")
import webbrowser
webbrowser.open(f"https://{endpoints['ui']}")

If successful, you should see the NGINX success page in your browser when visiting the endpoint.

Next, suspend the compute pool and service:

In [ ]:
new_compute_pool_def.suspend()
nginx_service.suspend()

Finally, delete the compute pool and service:

In [ ]:
new_compute_pool_def.delete()
nginx_service.delete()

Close your session with Snowflake:

In [ ]:
session.close()

Congratulations! In this notebook, you learned the fundamentals for managing Snowpark Container Services using the Snowflake Python API.

For more information, see the following resources:

* [Snowflake Documentation: Snowflake Python API](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/snowflake-python-overview)

* [Snowflake Documentation: Snowpark Container Services](https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overview)

* [Snowflake Python API Reference Documentation](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/reference/0.1.0/index.html)